# All Transactions

## Import All Transactions Data from GSheet

In [ ]:
import pandas as pd
import numpy as np
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

os.chdir(r'C:\Users\luc57.DESKTOP-NB5DC80\AE\ipynb')
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# here enter the id of your google sheet
SAMPLE_SPREADSHEET_ID_input = ''
SAMPLE_RANGE_NAME = 'A1:ZZ25000'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES) # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

main()

df_all=pd.DataFrame(values_input[1:], columns=values_input[0])
#file must be a google sheets, not a normal xlsx uploaded to gdrive

In [ ]:
len(df_all) #check the total number of rows

In [ ]:
df_all #get a glimpse of the head and tail of the data

## Data Transformation

In [ ]:
df_all.shape #there are 10236 rows and 54 columns

In [ ]:
df_all.columns #the column names

In [ ]:
df_all.dtypes #data types of the df_all columns

Columns _gross_sales_ to _returned_item_quantity_ are all numeric.

## Convert column datatypes

In [ ]:
#convert the columns to numeric
cols = ['orders','gross_sales','discounts','returns','net_sales','shipping','taxes',
        'total_sales','total_tips','average_order_value','total_cost','gross_profit',
        'gross_margin','units_per_transaction','net_quantity','ordered_item_quantity','returned_item_quantity']
df_all[cols] = df_all[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df_all.dtypes

## Correct the day variable

In [ ]:
#convert the day variable data type
from datetime import datetime
df_all['hour'] = pd.to_datetime(df_all['hour'])
df_all['day'] = df_all['hour'] 

In [ ]:
#remove the date from hour 
df_all['hour'] = df_all['hour'].apply(lambda x: x.strftime('%H:%M:%S'))
df_all['hour']

In [ ]:
for column in df_all.columns:
     print("\n" + column)
     print(df_all[column].value_counts())

## Filter only real orders

In [ ]:
df_all=df_all.loc[df_all['orders']>=1]

In [ ]:
df_all = df_all.loc[df_all['net_quantity']>=1]
len(df_all)

## Filter refunded items

In [ ]:
df_all.financial_status.value_counts()

In [ ]:
df_all=df_all.loc[df_all['financial_status']!='refunded']

Variables to pay attention to: sale_line_type (product or gift card), cancelled (Yes or No),
financial status (paid,refunded, partially refunded, pending),fulfillment status (fulfilled, unfulfilled),
referrer source (direct, unknown, search, social, email).

## Drop useless columns 

In [ ]:
df_all.drop(columns=['adjustment','sale_kind','orders','returns','shipping','total_tips',
                       'total_cost','gross_profit','gross_margin',
                        'units_per_transaction','net_quantity','ordered_item_quantity',
                        'returned_item_quantity'],inplace=True)
                        #all columns only have one value

In [ ]:
df_all.loc[df_all['net_sales']==0][['day','order_name','customer_name','product_title','variant_title',
                                  'total_sales','net_sales']]

In [ ]:
df_all[df_all['order_name']=='#1645'][['day','order_name','order_id','total_sales','discounts']]

In [ ]:
df_all.loc[df_all['order_name']=='#1645'][['billing_country', 'billing_postal_code',
       'customer_email', 'customer_id', 'customer_name', 'customer_type',
       'utm_campaign_content', 'utm_campaign_medium', 'utm_campaign_name',
       'utm_campaign_source', 'utm_campaign_term', 'product_id',
       'product_price', 'product_title', 'product_type']]

In [ ]:
df_all.loc[df_all['order_name']=='#1645'][['variant_sku',
       'variant_id', 'variant_title', 'api_client_title', 'shipping_city',
       'shipping_country', 'shipping_postal_code', 'referrer_host',
       'referrer_name', 'referrer_path', 'referrer_source', 'referrer_url',
       'gross_sales', 'discounts', 'net_sales']]

Every row is segregated into each product level. So, one order number may have 2 rows if the customer purchased two products. If a customer bought 4 products, the dataframe contains 4 of the same order names.

## Rename _total_sales_ to _sales_after_tax_

In [ ]:
df_all.rename(columns={"total_sales": "sales_before_tax"},inplace=True)

# Discount Data

## Import Discount Data from GSheet

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

os.chdir(r'C:\Users\luc57.DESKTOP-NB5DC80\AE\ipynb')
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# here enter the id of your google sheet
SAMPLE_SPREADSHEET_ID_input = ''
SAMPLE_RANGE_NAME = 'A1:ZZ5000'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES) # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

main()

discount_df=pd.DataFrame(values_input[1:], columns=values_input[0])
#file must be a google sheets, not a normal xlsx uploaded to gdrive

In [ ]:
discount_df

In [ ]:
#discount_df.set_index(df.columns[0])
#discount_df=discount_df.drop([''],axis=1) #drop the duplicate index column

## Discount Data Transformation

In [ ]:
discount_df.shape

In [ ]:
discount_df.columns

In [ ]:
discount_df.dtypes

Columns _orders_ to _total_quantity_ are all numeric.

## Convert column datatypes

In [ ]:
discount_df.columns

In [ ]:
#convert the columns to numeric
cols_discount_df = ['orders', 'total_gross_sales', 'total_discount_amount',
       'total_returns', 'total_net_sales', 'total_shipping_price',
       'total_shipping_discount', 'total_tax_amount', 'grand_total_sales',
       'total_line_item_script_discount', 'total_shipping_script_discount',
       'total_product_price', 'total_quantity_returns', 'total_quantity']
discount_df[cols_discount_df] = discount_df[cols_discount_df].apply(pd.to_numeric, errors='coerce', axis=1)
discount_df.dtypes

## Create new variable _date_ and correct the _hour_ variable

In [ ]:
discount_df[['hour','day','week','month','quarter','year']]

In [ ]:
#create new variable date from the hour variable
from datetime import datetime
discount_df['hour'] = pd.to_datetime(discount_df['hour'])
discount_df['date'] = discount_df['hour']

#remove the date from hour 
discount_df['hour'] = discount_df['hour'].apply(lambda x: x.strftime('%H:%M:%S'))
discount_df['hour']

## Filter only transactions that have monetary value

In [ ]:
discount_df=discount_df.loc[discount_df['total_gross_sales']>0]
len(discount_df)

## Value Distribution of the Columns

In [ ]:
for column in discount_df.columns:
     print("\n" + column)
     print(discount_df[column].value_counts())

In [ ]:
discount_df = discount_df.drop(columns=['name','discount_applied','automatic_discount_title',
                                        'marketing_event_target','product_vendor','shipping_title',
                                        'marketing_event_type','marketing_event_target',
                                        'total_shipping_price','total_shipping_discount',
                                        'total_line_item_script_discount',
                                        'total_shipping_script_discount'])

## Rename column names

In [ ]:
print(len(discount_df.columns)) #check how many columns are remaining
discount_df.columns

In [ ]:
discount_df.rename(columns={"total_gross_sales": "gross_sales",
                            "total_discount_amount":"discounts",
                            "total_returns":"returns (in EUR)",
                            "total_net_sales":"net_sales",
                            "total_tax_amount":"taxes",
                            "total_product_price":"product_price",
                            "total_quantity_returns":"quantity_returns",
                            "total_quantity":"quantity"},inplace=True)

In [ ]:
discount_df[['gross_sales','discounts','returns (in EUR)','grand_total_sales','taxes','net_sales']]
#gross sales = product price x quantity (before taxes, shipping, discounts, and returns)
#grand total sales = gross sales - discounts - returns + taxes + shipping charges
#net sales = gross sales - discounts - returns + shipping + taxes
#shipping = shipping charges - shipping discounts - refunded shipping amounts

# Add discount code to the bigger dataframe

In [ ]:
print(len(df_all))
print(len(discount_df))

In [ ]:
discount_df_selection=discount_df[['order_name','discount_code']]
discount_df_selection = discount_df_selection.drop_duplicates()
df_all_in = pd.merge(df_all,discount_df_selection,on='order_name',how='left')
df_all_in.sort_values(by='day',ascending=False)

### Convert customer name to all caps lock

In [ ]:
df_all_in['customer_name']=df_all_in['customer_name'].apply(lambda x: x.upper())

# Campaigns in 2019

In [ ]:
import pandas as pd
df_all_in = pd.read_csv('df.csv')

## Earlybird

In [ ]:
Earlybird = df_all_in.loc[df_all_in['discount_code']=='Earlybird']
Earlybird

In [ ]:
Earlybird.columns

###  Number of Unique Orders, Number of Unique Customers, Customer Names

In [ ]:
Earlybird.groupby('order_name')[['gross_sales','discounts','taxes','net_sales']].sum()

In [ ]:
print('Number of Unique Orders: ',len(Earlybird.order_name.unique())) #number of unique orders, excl. returns
print('Number of Unique Customers: ',len(Earlybird.customer_name.unique())) #number of unique customers
Earlybird_customer_list=Earlybird.customer_name.unique().tolist() #convert the numpy array of cust. names to a list
print('Customer List: ', Earlybird_customer_list)

In [ ]:
obtained_customers=ARI20_customer_list + ARI50_customer_list
print(obtained_customers)
[i for i in obtained_customers if i in Earlybird_customer_list] #check if customers in Earlybird
                                                                  #have purchased before

In [ ]:
print('Number of Products Purchased by Each Customer:','\n', Earlybird.customer_name.value_counts()) 

### Gross Sales, Discounts, Net Sales

In [ ]:
Earlybird[['gross_sales','discounts','net_sales']].agg('sum')

### Net Sales of Each Customer who used ARI50

In [ ]:
Orders_of_Earlybird_customers=df_all_in.loc[df_all_in['customer_name'].isin (Earlybird_customer_list)]
#orders by each Earlybird customers
Orders_of_Earlybird_customers.groupby('customer_name').sum()['net_sales'].sort_values(ascending=False)

### Additional Quantity and Revenue generated

In [ ]:
Earlybird_df_all=Orders_of_Earlybird_customers[['order_name','discount_code','day','customer_name','net_sales']]
Earlybird_df_all.sort_values(by='customer_name')[:10]

In [ ]:
Earlybird_df_all.groupby('customer_name')['order_name'].nunique()

In [ ]:
orders_by_customers=Earlybird_df_all.groupby('customer_name')['order_name'].nunique()
orders_by_customers.where(orders_by_customers > 1)

In [ ]:
list = ['']
Earlybird_df_all[Earlybird_df_all['customer_name']. isin(list)].sort_values(by='day',ascending=False)

In [ ]:
Earlybird_cust = Earlybird_df_all[Earlybird_df_all['customer_name']. isin(list)]
Earlybird_return = Earlybird_cust.loc[(Earlybird_cust['day']>'2019-06-04')&
                                      (Earlybird_cust['net_sales']>0) &
                                      (Earlybird_cust['discount_code']!='Earlybird')]
print('Number of additional orders: ',len(Earlybird_return.groupby('order_name')))
print('Additional net sales: ',Earlybird_return.net_sales.sum())

In [ ]:
Earlybird_return

In [ ]:
Earlybird_df_all.loc[Earlybird_df_all['customer_name']==input()][['day','order_name','discount_code','net_sales']].sort_values(by=
                                                                                                                         'day',ascending=False)

# Number of returners

## Number of Returning Customers

In [ ]:
returning_customers=df_new.loc[df_new['customer_type']=='Returning']
returning_customers.customer_name.value_counts() #there are 434 returning customers

## Campaigns with the highest number of returners

In [ ]:
#calculate return rates
print('Total Return Rate: ', round(len(returning_customers.customer_name.unique())/
     len(df_new.customer_name.unique())*100),'%')

In [ ]:
#return rates for campaigns in 2019
print('Number of unique returning customers in Earlybird: ',
      "{0:.0%}".format(len(returning_customers.loc[returning_customers['discount_code']=='Earlybird'].
          customer_name.unique())/len(Earlybird.customer_name.unique())))

In [ ]:
returning_customers.discount_code.value_counts()[:30]

In [ ]:
df_new.columns

In [ ]:
df_new[['referrer_host','referrer_name','referrer_path','referrer_source','referrer_url']].nunique()

In [ ]:
df_new['referrer_name'].value_counts()

In [ ]:
df_new.order_name.nunique()

# SQL

In [ ]:
import sqlite3
conn = sqlite3.connect('my_database.sqlite')
cursor = conn.cursor()
print("Opened database successfully")

In [ ]:
fields = "hour, cancelled, financial_status, fulfillment_status,order_id, order_name, sale_line_type, \
       cost_tracked,billing_company, billing_city, billing_region, billing_country, \
       billing_postal_code, customer_email, customer_id, customer_name, \
       customer_type, marketing_event_target, marketing_event_type, \
       utm_campaign_content, utm_campaign_medium, utm_campaign_name, \
       utm_campaign_source, utm_campaign_term, pos_location_name, \
       product_id, product_price, product_title, product_type, \
       product_vendor, variant_id, variant_sku, variant_title, \
       api_client_title, shipping_city, shipping_region, \
       shipping_country, shipping_postal_code, referrer_host, \
       referrer_name, referrer_path, referrer_source, referrer_url, \
       gross_sales, discounts, net_sales, taxes, sales_before_tax, \
       average_order_value, day, discount_code, disc_is_null" 
query = f"create TABLE CUST_DATA ({fields})"
cursor.execute(query)

In [ ]:
df_new.to_sql('sales', conn, if_exists='append', index=False)

In [ ]:
rows = cursor.execute("SELECT * FROM sales \
                      WHERE discount_code = 'BF19'").fetchall()
pd.DataFrame(rows, columns=[i[0] for i in cursor.description])